In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from model.classification import *
from src import *
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from ipywidgets import interact, widgets

In [3]:
model, config = load_model_locally('one_location', 'rvce')

In [4]:
files = [
    '12_RX100',
    '12_iphone',
    '12_Mobius'
]

datapool = DataPool(files, config.window_length, config.split_ratio, tqdm=tqdm)

Video loading:   0%|          | 0/3 [00:00<?, ?it/s]

In [82]:
config_1 = EasyDict(config)
config_2 = EasyDict(config)

In [83]:
config_1.sr = config_2.sr = 22050
# config_2.normalization = Normalization.ROW_WISE
# config_2.top_db = 20
config_1.sr, config_2.sr
config_2.top_db = -1

In [84]:
transform_1 = create_transformation(config_1)
transform_2 = create_transformation(config_2)

file_number = 1
signal = datapool[files[file_number]].signal.clone()

def f(from_time=0):
    till_time = from_time + 10
    
    from_sample = from_time * config.sr
    till_sample = till_time * config.sr
    
    signal_1 = signal[from_sample: till_sample].clone()
    features_1 = transform_1(signal_1).squeeze()

    signal_2 = signal[from_sample: till_sample].clone()
    features_2 = transform_2(signal_2).squeeze()
        
    vmin = min(features_1.min(), features_2.min())
    vmax = max(features_1.max(), features_2.max())
    
    fig, axes = plt.subplots(ncols=2, figsize=(15,5))
    ax1, ax2 = axes.flatten()
        
    im1 = ax1.pcolormesh(features_1, vmin=vmin, vmax=vmax)
    im2 = ax2.pcolormesh(features_2, vmin=vmin, vmax=vmax)

    plt.tight_layout()
    fig.colorbar(im2, ax=[ax1, ax2])
    plt.show()

from_time_slider = widgets.IntSlider(min=0, max=60*20, step=1, value=260)
interact(f, from_time=from_time_slider);

interactive(children=(IntSlider(value=260, description='from_time', max=1200), Output()), _dom_classes=('widge…

In [81]:
from_time = 280
till_time = from_time + 6

from_sample = from_time * config.sr
till_sample = till_time * config.sr

signal = datapool[files[file_number]].signal

signal_1 = signal[from_sample: till_sample].clone()
features_1 = transform(signal_1).squeeze()

signal_maes = []
feature_maes = []
volumes = np.arange(1, 50, .2)

for volume in volumes:
    volume_transform = torchaudio.transforms.Vol(volume, 'db')

    signal_2 = signal[from_sample: till_sample].clone()
    signal_2 = volume_transform(signal_2)
    features_2 = transform(signal_2).squeeze()
    
    signal_mae = (signal_1 - signal_2).abs().mean().item()
    feature_mae = (features_1 - features_2).abs().mean().item()
    

    signal_maes.append(signal_mae)
    feature_maes.append(feature_mae)

fig, axes = plt.subplots(1, 2, figsize=(12,5))
axes[0].set_xlabel('Volume')
axes[0].set_ylabel('MAE signal')

axes[1].set_xlabel('Volume')
axes[1].set_ylabel('MAE features')
axes[0].plot(volumes, signal_maes);
axes[1].plot(volumes, feature_maes);

NameError: name 'transform' is not defined